In [8]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Add
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy
import scipy.misc
import os
from PIL import Image

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [9]:
def label(filename):
  if 'virus' in filename  or 'bacteria' in filename:
    return 1
  else:
    return 0


def image2vector(path, filename, num_px = 128):
  filename = path + filename
  new_image = Image.open(filename).convert('RGB').resize((num_px,num_px))
  new_image = np.array(new_image)
  return new_image

def load_data(path, num_px):
  X = []
  Y = []
  
  for entry in os.scandir(path):
    if entry.is_file():
       X.append(image2vector(path, entry.name, num_px))
       Y.append([label(entry.name)])

  X = np.array(X)
  Y = np.array(Y).T
  
  return X, Y

In [10]:
path_train = 'D:/Documentos/deep_learning/trabajos/Neumonia/chestxray/train/'
path_test = 'D:/Documentos/deep_learning/trabajos/Neumonia/chestxray/test/'
path_val = 'D:/Documentos/deep_learning/trabajos/Neumonia/chestxray/val/'

In [11]:
X_train_orig, Y_train_orig = load_data(path_train, 128)

In [12]:
X_test_orig, Y_test_orig = load_data(path_test, 128)

In [13]:
x_val, y_val = load_data(path_val, 128)

In [6]:
# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 5216
number of test examples = 624
X_train shape: (5216, 128, 128, 3)
Y_train shape: (5216, 1)
X_test shape: (624, 128, 128, 3)
Y_test shape: (624, 1)


In [7]:
def pneumonia (input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='pneumonia')   
    
    return model

In [15]:
def identity_block(X, f, filters, stage, block):   
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
       
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [16]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides = (1,1), padding='same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides = (1,1), padding='valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), padding='valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [43]:
def pneumonia_rnn(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='pneumonia_rnn')

    return model

In [8]:
# model create
pneumonia = pneumonia(X_train.shape[1:])

In [9]:
# model compile
pneumonia.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [12]:
# model train
pneumonia.fit(X_train, Y_train, epochs=20, batch_size=200)

Epoch 1/20
27/27 [==============================] - 81s 3s/step - loss: 0.0662 - accuracy: 0.9793
Epoch 2/20
27/27 [==============================] - 82s 3s/step - loss: 0.0360 - accuracy: 0.9850
Epoch 3/20
27/27 [==============================] - 82s 3s/step - loss: 0.0223 - accuracy: 0.9916
Epoch 4/20
27/27 [==============================] - 82s 3s/step - loss: 0.0216 - accuracy: 0.9912
Epoch 5/20
27/27 [==============================] - 82s 3s/step - loss: 0.3229 - accuracy: 0.9392
Epoch 6/20
27/27 [==============================] - 81s 3s/step - loss: 0.0964 - accuracy: 0.9760
Epoch 7/20
27/27 [==============================] - 81s 3s/step - loss: 0.0382 - accuracy: 0.9872
Epoch 8/20
27/27 [==============================] - 80s 3s/step - loss: 0.0413 - accuracy: 0.9849
Epoch 9/20
27/27 [==============================] - 82s 3s/step - loss: 0.0425 - accuracy: 0.9856
Epoch 10/20
27/27 [==============================] - 82s 3s/step - loss: 0.0177 - accuracy: 0.9927
Epoch 11/20
27/27 [

In [13]:
# model evaluate
preds = pneumonia.evaluate(X_test, Y_test, batch_size = 50, verbose = 1, sample_weight=None)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

13/13 [==============================] - 1s 109ms/step - loss: 3.3132 - accuracy: 0.6987

Loss = 3.313183069229126
Test Accuracy = 0.6987179517745972


In [31]:
# model summary
pneumonia.summary()

Model: "pneumonia"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 134, 134, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 128, 128, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 128, 128, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 64, 64, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0 

In [44]:
# model RNN create
model_rnn = pneumonia_rnn(input_shape = (128, 128, 3), classes = 1)

In [45]:
#model RNN compile
model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
# model RNN train
model_rnn.fit(X_train, Y_train, epochs = 20, batch_size = 500)

Epoch 1/20
11/11 [==============================] - 724s 66s/step - loss: 8.8561e-08 - accuracy: 0.6593
Epoch 2/20
11/11 [==============================] - 652s 59s/step - loss: 8.8561e-08 - accuracy: 0.6582
Epoch 3/20
11/11 [==============================] - 702s 64s/step - loss: 8.8561e-08 - accuracy: 0.6559
Epoch 4/20
11/11 [==============================] - 720s 65s/step - loss: 8.8561e-08 - accuracy: 0.6561
Epoch 5/20
11/11 [==============================] - 650s 59s/step - loss: 8.8561e-08 - accuracy: 0.6536
Epoch 6/20
11/11 [==============================] - 651s 59s/step - loss: 8.8561e-08 - accuracy: 0.6601
Epoch 7/20
11/11 [==============================] - 677s 62s/step - loss: 8.8561e-08 - accuracy: 0.6589
Epoch 8/20
11/11 [==============================] - 714s 65s/step - loss: 8.8561e-08 - accuracy: 0.6543
Epoch 9/20
11/11 [==============================] - 703s 64s/step - loss: 8.8561e-08 - accuracy: 0.6645
Epoch 10/20
11/11 [==============================] - 693s 63s/st

In [47]:
# model RNN evaluate
preds_rnn = model_rnn.evaluate(X_test, Y_test)
print ("Loss = " + str(preds_rnn[0]))
print ("Test Accuracy = " + str(preds_rnn[1]))

20/20 [==============================] - 9s 463ms/step - loss: 7.4506e-08 - accuracy: 0.6250
Loss = 7.450580596923828e-08
Test Accuracy = 0.625


In [48]:
# model RNN summary
model_rnn.summary()

Model: "pneumonia_rnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 134, 134, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
______________________________________________________________________________________

In [55]:
# model exports
pneumonia.save('pneumonia_convnet.h5')
model_rnn.save('pneumonia_resnet.h5')

In [ ]:
# model import ConvNet
pneumonia_convnet = load_model('pneumonia_convnet.h5')

In [ ]:
# model import ResNet
pneumonia_resnet = load_model('pneumonia_resnet.h5')

In [ ]:
# model pneumonia_convnet val
# specify img_path
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(128, 128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("Prediction: ")
print(pneumonia_convnet.predict(x))

In [ ]:
# model pneumonia_resnet val
# specify img_path
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(128, 128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("Prediction: ")
print(pneumonia_resnet.predict(x))